In [ ]:
from prot_db import bigtable_constants as btc, index_seq_files
import itertools
import os
from utils import file_util, notebook_util, fasta_util, ggdb_logging
import pandas as pd
import sys
import pprint
import subprocess

import importlib
importlib.reload(fasta_util)


pd.set_option('display.max_colwidth', None)
notebook_util.disp_notebook_full_width()

In [ ]:
# SCRAPE_DIR = "/GeneGraphDB/data/mgnify_scrape_20220505"
SCRAPE_DIR = "gs://durrant_prot_pred_db/mgnify_scrape_20220505"

faa_glob = os.path.join(SCRAPE_DIR, "**/*.faa.gz")
anno_glob = os.path.join(SCRAPE_DIR, "**/*annotations.gff.bgz")

faa_files = file_util.glob(faa_glob)
anno_files = file_util.glob(anno_glob)



In [ ]:
len(faa_files), len(anno_files)

In [ ]:
faa_files[0], anno_files[0]

df_faa = pd.DataFrame(faa_files, columns=["path"])
df_faa["type"] = "faa"

df_anno = pd.DataFrame(anno_files, columns=["path"])
df_anno["type"] = "annotation"


df_combo = pd.concat([df_faa, df_anno])
df_combo

skipped_dirs = 0

df_combo["dirname"] = df_combo["path"].map(os.path.dirname)
for dirname, df_dir in df_combo.groupby("dirname"):
    if len(df_dir) != 2:
        skipped_dirs += 1
        continue
    count_by_type = df_dir["type"].value_counts()
    if count_by_type['faa'] != 1 or count_by_type['annotation'] != 1:
        skipped_dirs += 1
        continue
        
    fasta_fpath = df_dir[df_dir["type"] == "faa"].path.values[0]
    annotation_fpath = df_dir[df_dir["type"] == "annotation"].path.values[0]
    
    filenames.append((fasta_fpath, annotation_fpath))
    
    
        
print("Skipped dirs", skipped_dirsd_dirs)



In [ ]:
df_dir[df_dir["type"] == "faa"].path.values[0]

In [ ]:
df_dir["type"].value_counts()['faa']

In [ ]:
faa_fpath = "/GeneGraphDB/data/mgnify_scrape_20220505/studies/MGYS00002012/samples/ERS433542/analyses/MGYA00598832/ERZ1746111_FASTA_predicted_cds.faa.gz"
anno_fpath = "/GeneGraphDB/data/mgnify_scrape_20220505/studies/MGYS00002012/samples/ERS433542/analyses/MGYA00598832/ERZ1746111_FASTA_annotations.gff.bgz"

In [ ]:
# dummy_index faa_path anno_path

def _row_key_from_seq_record(seq_record):
    description = seq_record.description
    fields = description.split(" # ")
    seq_id, begin_idx, end_idx = fields[0:3]
    # e.g. ERZ1746111.1-NODE-1-length-464040-cov-12.638829_3 -> ERZ1746111.1-NODE-1-length-464040-cov-12.638829
    non_indexed_seq_id = seq_id.split("_", -1)[0]
    seq_id_start_end = "@".join((non_indexed_seq_id, begin_idx, end_idx))
    return seq_id_start_end

def _row_key_from_gff_row(gff_row):
    seq_id_start_end = "@".join((gff_row.seqid, gff_row.start, gff_row.end))
    return seq_id_start_end
    

In [ ]:
gff_rows = fasta_util.parse_gff_file(anno_fpath)
attribute_dict = {_row_key_from_gff_row(row): row.attributes for row in gff_rows}

In [ ]:
has_anno = []    
for seq in fasta_util.open_fasta(faa_fpath):
    if " # " not in seq.description:
        continue
    row_key = _row_key_from_seq_record(seq)
    has_anno.append(row_key in attribute_dict)

In [ ]:
print(len(has_anno))
print(sum(has_anno))

In [ ]:
list(attribute_dict.keys())[0:5]